In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
PATH = "data/house-prices-advanced-regression-techniques/"

In [4]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)

In [5]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [6]:
display_all(df_raw.tail().T)

1455     1456     1457     1458     1459
Id                1456     1457     1458     1459     1460
MSSubClass          60       20       70       20       20
MSZoning            RL       RL       RL       RL       RL
LotFrontage         62       85       66       68       75
LotArea           7917    13175     9042     9717     9937
Street            Pave     Pave     Pave     Pave     Pave
Alley              NaN      NaN      NaN      NaN      NaN
LotShape           Reg      Reg      Reg      Reg      Reg
LandContour        Lvl      Lvl      Lvl      Lvl      Lvl
Utilities       AllPub   AllPub   AllPub   AllPub   AllPub
LotConfig       Inside   Inside   Inside   Inside   Inside
LandSlope          Gtl      Gtl      Gtl      Gtl      Gtl
Neighborhood   Gilbert   NWAmes  Crawfor    NAmes  Edwards
Condition1        Norm     Norm     Norm     Norm     Norm
Condition2        Norm     Norm     Norm     Norm     Norm
BldgType          1Fam     1Fam     1Fam     1Fam     1Fam
HouseStyle      2Story   1Story   2Story   1Story   1Story
OverallQual          6        6        7        5        5
OverallCond          5        6        9        6        6
YearBuilt         1999     1978     1941     1950     1965
YearRemodAdd      2000     1988     2006     1996     1965
RoofStyle        Gable    Gable    Gable      Hip    Gable
RoofMatl       CompShg  CompShg  CompShg  CompShg  CompShg
Exterior1st    VinylSd  Plywood  CemntBd  MetalSd  HdBoard
Exterior2nd    VinylSd  Plywood  CmentBd  MetalSd  HdBoard
MasVnrType        None    Stone     None     None     None
MasVnrArea           0      119        0        0        0
ExterQual           TA       TA       Ex       TA       Gd
ExterCond           TA       TA       Gd       TA       TA
Foundation       PConc   CBlock    Stone   CBlock   CBlock
BsmtQual            Gd       Gd       TA       TA       TA
BsmtCond            TA       TA       Gd       TA       TA
BsmtExposure        No       No       No       Mn       No
BsmtFinType1       Unf      ALQ      GLQ      GLQ      BLQ
BsmtFinSF1           0      790      275       49      830
BsmtFinType2       Unf      Rec      Unf      Rec      LwQ
BsmtFinSF2           0      163        0     1029      290
BsmtUnfSF          953      589      877        0      136
TotalBsmtSF        953     1542     1152     1078     1256
Heating           GasA     GasA     GasA     GasA     GasA
HeatingQC           Ex       TA       Ex       Gd       Gd
CentralAir           Y        Y        Y        Y        Y
Electrical       SBrkr    SBrkr    SBrkr    FuseA    SBrkr
1stFlrSF           953     2073     1188     1078     1256
2ndFlrSF           694        0     1152        0        0
LowQualFinSF         0        0        0        0        0
GrLivArea         1647     2073     2340     1078     1256
BsmtFullBath         0        1        0        1        1
BsmtHalfBath         0        0        0        0        0
FullBath             2        2        2        1        1
HalfBath             1        0        0        0        1
BedroomAbvGr         3        3        4        2        3
KitchenAbvGr         1        1        1        1        1
KitchenQual         TA       TA       Gd       Gd       TA
TotRmsAbvGrd         7        7        9        5        6
Functional         Typ     Min1      Typ      Typ      Typ
Fireplaces           1        2        2        0        0
FireplaceQu         TA       TA       Gd      NaN      NaN
GarageType      Attchd   Attchd   Attchd   Attchd   Attchd
GarageYrBlt       1999     1978     1941     1950     1965
GarageFinish       RFn      Unf      RFn      Unf      Fin
GarageCars           2        2        1        1        1
GarageArea         460      500      252      240      276
GarageQual          TA       TA       TA       TA       TA
GarageCond          TA       TA       TA       TA       TA
PavedDrive           Y        Y        Y        Y        Y
WoodDeckSF           0      349        0      366      736
OpenPo

In [7]:
train_cats(df_raw)

In [9]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/house_prices_raw')

In [10]:
df_raw = pd.read_feather('tmp/house_prices_raw')

In [11]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [12]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.97569203774249069

In [13]:
p = pd.read_csv(f'{PATH}test.csv', low_memory=False)
apply_cats(p, df_raw)
pred, _, nas = proc_df(p)

In [14]:
df.shape, pred.shape

((1460, 83), (1459, 91))

In [15]:
pred.drop(['BsmtFinSF1_na','BsmtFinSF2_na','BsmtUnfSF_na','TotalBsmtSF_na','BsmtFullBath_na','BsmtHalfBath_na','GarageCars_na','GarageArea_na'], axis=1, inplace=True)

In [16]:
df.shape, pred.shape

((1460, 83), (1459, 83))

In [17]:
new_preds = m.predict(pred)

In [19]:
pd.DataFrame({'Id':pred.Id, 'SalePrice':new_preds}).set_index('Id').to_csv('predictions/house_prices/sample_submission.csv')

In [20]:
new_preds

array([ 131230.,  151950.,  181770., ...,  156340.,   99600.,  227629.])